# Finding Hidden Message in DNA
## Week 3

**Randomized Algorithms** <br>
Will use random decisions to search for motifs.
* Las Vegas Algorithms - deliver solutions guaranteed to be exact, despite reliance on random decisions
* Monte Carlo Algorithms - Quickly find approximate solutions, not guaranteed to be correct

In [1]:
# Helper Functions from previous weeks
def symToNum(symbol):
    symMap = {'A': 0,
              'C': 1,
              'G': 2,
              'T': 3}
    return symMap[symbol]

def numToSym(num):
    numMap = {0: 'A',
              1: 'C',
              2: 'G',
              3: 'T'}
    return numMap[num]

def numToPat(index, k):
    if k == 1:
        return numToSym(index)
    pfixIndex = int(index / 4)
    r = index % 4
    sym = numToSym(r)
    pfixPattern = numToPat(pfixIndex, k - 1)
    return pfixPattern + sym

def patToNum(pattern):
    if not pattern:
        return 0
    symbol = pattern[-1]
    prefix = pattern[:-1]
    return 4 * patToNum(prefix) + symToNum(symbol)

def hammDist(seq1, seq2, d=-1):
    dist = 0
    if len(seq1) != len(seq2):
        return -1
    for i in range(len(seq1)):
        if seq1[i] != seq2[i]:
            dist += 1
        if d != -1 and dist > d:
            return -1
    return dist

def profileMost(Text, k, Profile):
    profMax = 0.0
    kmerMax = Text[:k]
    for i in range(len(Text) - k + 1):
        kmer = Text[i:i+k]
        tProf = 1.0
        for x in range(k):
            tProf *= Profile[symToNum(kmer[x])][x]
        if tProf > profMax:
            profMax = tProf
            kmerMax = kmer
    return kmerMax

def makeProfile(Dna, pCounts=True):
    prof = [[0] * len(Dna[0]) for _ in range(4)]
    for string in Dna:
        for i in range(len(string)):
            prof[symToNum(string[i])][i] += 1
    if pCounts:
        return [[(x+1)/(len(Dna) + 4) for x in i] for i in prof]
    else:
        return [[x/t for x in i] for i in prof]

def makeCon(prof):
    con = ''
    test = list(zip(*prof))
    for pos in test:
        con += numToSym(pos.index(max(pos)))
    return con

def score(motifs, pCounts=True):
    prof = makeProfile(motifs, pCounts)
    con = makeCon(prof)
    score = [hammDist(con, x) for x in motifs]
    return sum(score)

## Randomized Motif Search

1. We utilize a function that generates a random selection of k-mers (motif) from our Dna strings
2. We calculate the profile of this motif
3. We generate a new motif, for each string in Dna we calculate the profile most probable k-mer
4. We recalculate the profile & score and continue iterating as long as the score continues to decrease

We expect the profile of a randomly selected motif to be a uniform profile (all probabilities 1/4) compared to the opposite which is randomly choosing the implanted k-mers correctly the first time resulting in our optimal profile. In practice we obtain a profile somewhere in the middle. Choosing a random motif and iterating allows us to direct the profile away from the uniform toward the ideal profile based off the implanted motif

In [2]:
# Randomized Motif Search
import random as rd

def randMotif(Dna, k):
    rd.seed()
    ran = rd.randint(0,len(Dna[0]) - k)
    res = []
    for seq in Dna:
        res.append(seq[ran:ran+k])
    return res

def mostProb(tProf, Dna, k):
    newMotif = []
    for seq in Dna:
        newMotif.append(profileMost(seq, k, tProf))
    return newMotif

def randMotifSearch(Dna, k, t):
    gMotif = randMotif(Dna, k)
    gScore = score(gMotif)
    tMotif = gMotif
    while True:
        tProf = makeProfile(tMotif)
        tMotif = mostProb(tProf, Dna, k)
        tScore = score(tMotif)
        if tScore < gScore:
            gScore = tScore
            gMotif = tMotif
        else:
            return gMotif

def randMotifDrive(Dna, k, t, N):
    bestMotif = randMotifSearch(Dna, k, t)
    bestScore = score(bestMotif)
    for _ in range(N):
            tMotif = randMotifSearch(Dna, k, t)
            tScore = score(tMotif)
            if tScore < bestScore:
                bestScore = tScore
                bestMotif = tMotif
    return bestMotif

# Failing Debug Tests
#6, 8
#dna = ['GCACATCATTAAACGATTCGCCGCATTGCCTCGATTAACC', 'TCATAACTGACACCTGCTCTGGCACCGCTCATCCAAGGCC', 'AAGCGGGTATAGCCAGATAGTGCCAATAATTTCCTTAACC', 'AGTCGGTGGTGAAGTGTGGGTTATGGGGAAAGGCAAGGCC', 'AACCGGACGGCAACTACGGTTACAACGCAGCAAGTTAACC', 'AGGCGTCTGTTGTTGCTAACACCGTTAAGCGACGAAGGCC', 'AAGCTTCCAACATCGTCTTGGCATCTCGGTGTGTTTAACC', 'AATTGAACATCTTACTCTTTTCGCTTTCAAAAAAAAGGCC']

#8, 5
#dna = ['CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA', 'GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG', 'TAGTACCGAGACCGAAAGAAGTATACAGGCGT', 'TAGATCAAGTTTCAGGTGCACGTCGGTGAACC', 'AATCCACCAGCTCCACGTGCAATGTTGGCCTA']

#Working Debug Tests
## Beginning Off-by-one error test
# 6,8
dna = ['AATTGGCACATCATTATCGATAACGATTCGCCGCATTGCC', 'GGTTAACATCGAATAACTGACACCTGCTCTGGCACCGCTC', 'AATTGGCGGCGGTATAGCCAGATAGTGCCAATAATTTCCT', 'GGTTAATGGTGAAGTGTGGGTTATGGGGAAAGGCAGACTG', 'AATTGGACGGCAACTACGGTTACAACGCAGCAAGAATATT',  'GGTTAACTGTTGTTGCTAACACCGTTAAGCGACGGCAACT', 'AATTGGCCAACGTAGGCGCGGCTTGGCATCTCGGTGTGTG', 'GGTTAAAAGGCGCATCTTACTCTTTTCGCTTTCAAAAAAA']
randMotifDrive(dna, 6, 8, 1000)

# with open("./Data/dataset_161_5.txt") as inFile:
#     data = inFile.readlines()
    
# args = data[0].strip().split(' ')
# k = int(args[0])
# t = int(args[1])
# dna = []
# for i in range(1, len(data)):
#     dna.append(data[i].strip())
# print(*randMotifDrive(dna, k ,t),sep="\n")

['CGATAA',
 'GGTTAA',
 'GGTATA',
 'GGTTAA',
 'GGTTAC',
 'GGTTAA',
 'GGCCAA',
 'GGTTAA']

In [3]:
#Compute the probability that ten randomly selected 15-mers from the ten 600-nucleotide long sequences
# capture at least one implanted 15-mer.

#P(at least 1 implanted k-mer) = 1 - P(0 implanted k-mers)

#P(0 implanted k-mers in 10 strings)
print(600 - 15 + 1)
# = 586
#so out of 586 possible k-mer start locations only 1 is correct

# For one string the probability of choosing incorrect k-mer is 
print(585/586)
# = 0.9983

# For 10 strings this is
print((585/586) ** 10)
# = 0.9831
#So our answer is 1 minus that probability
print(1-((585/586) ** 10))

586
0.9982935153583617
0.9830656030708897
0.01693439692911025


In [4]:
#Compute the probability that ten randomly selected 15-mers from ten 600-nucleotide long strings
# capture at least two implanted 15-mers.

# P(at least two implanted k-mer) = 1 - P(0 implanted k-mers) - P(1 implanted k-mer)
print((1/585) * ((585/586) ** 9))
print(1 - ((600-15)/(600-15+1))**10 - 10*(600-15)**9/(600-15+1)**10)

0.001683326593321766
0.00012985670567623384


In our randomized motif search all t strings in motif may change in a single iteration. This strategy might be potentially discarded. Utilizing a GibbsSampler, a more cautious approach that discards a single k-mer from the current set of motifs at each iteration.

To describe how a GibbsSampler updates Motifs, we will need a more advanced random number generator. Given a probability distribution the number generator models an n-sided die and returns integer i with probability $p_{i}$<br>
Gibbs Outline:
1. Form <em>Motifs</em> by randomly selecting a k-mer from each sequence
2. Randomly choose one of the selected k-mers and remove from <em>Motifs</em>
3. Create Profile from the remaining k-mers in <em>Motifs</em>
4. For each k-mer that was removed calculate 

We have defined a Profile-most probable k-mer in a string, now we define a Profile-randomly generated k-mer.

In [5]:
%%time
# Gibbs Sampler

def profileRand(Text, k, tProf):
    probs = []
    for i in range(len(Text) - k + 1):
        tProb = 1.0
        kmer = Text[i:i+k]
        for i, base in enumerate(kmer):
            tProb *= tProf[symToNum(base)][i]
        probs.append(tProb)
    probSum = sum(probs)
    probs = [x/probSum for x in probs]
    choice = rd.choices([x for x in range(len(probs))], weights=probs, k=1)
    return Text[choice[0]:choice[0]+k]

def gibbs(Dna, k, t, N):
    motifs = randMotif(Dna, k)
    bestMotif = motifs
    bestScore = score(bestMotif)
    for _ in range(N):
        i = rd.randint(0,t-1)
        tProf = makeProfile(motifs[:i] + motifs[i+1:])
        motifs[i] = profileRand(Dna[i], k, tProf)
        tScore = score(motifs)
        if tScore < bestScore:
            bestScore = tScore
            bestMotif = motifs[:]
    return bestMotif

def gibbsDrive(Dna, k, t, N):
    bestMotif = gibbs(Dna, k, t, N)
    bestScore = score(bestMotif)
    for _ in range(30):
        tMotif = gibbs(Dna, k, t, N)
        tScore = score(tMotif)
        if tScore < bestScore:
            bestScore = tScore
            bestMotif = tMotif
    return bestMotif

def gibbsDriveWithRandomMotif(Dna, k, t, N):
    bestMotif = randMotifDrive(Dna, k, t, int(N/4))
    bestScore = score(bestMotif)
    for _ in range(30):
        tMotif = gibbs(Dna, k, t, N)
        tScore = score(tMotif)
        if tScore < bestScore:
            bestScore = tScore
            bestMotif = tMotif
    return bestMotif

# dna = ['CGCCCCTCTCGGGGGTGTTCAGTAACCGGCCA',
#        'GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG',
#        'TAGTACCGAGACCGAAAGAAGTATACAGGCGT',
#        'TAGATCAAGTTTCAGGTGCACGTCGGTGAACC',
#        'AATCCACCAGCTCCACGTGCAATGTTGGCCTA']
# print(gibbsDrive(dna, 8, 5, 100))
# print(gibbsDriveWithRandomMotif(dna, 8, 5, 100))
# print(gibbsDriveImprove(dna, 8, 5, 100))


with open("./Data/dataset_163_4.txt") as inFile:
    data = inFile.readlines()

args = data[0].strip().split(' ')
k = int(args[0])
t = int(args[1])
N = int(args[2])
dna = []
for i in range(1, len(data)):
    dna.append(data[i].strip())
print(*gibbsDrive(dna, k, t, N), sep="\n")
# print(*gibbsDriveWithRandomMotif(dna, k, t, N), sep="\n")

CCATTTGGAAGGCGT
ATAGCCGGTCGGGTT
AGGGATGGTCGGGTT
ATATATCAACGGGTT
ATATATGGTGTTGTT
ATATATGAATGGGTT
ATATATTTCCGGGTT
GCGTATGGTCGGGTT
ATCGTTGGTCGGGTT
ATATATGGTCGGACG
ATATATGGTCCTCTT
ATATATGGGAAGGTT
ATAGGGGGTCGGGTT
GAATATGGTCGGGTC
ATATATGGTCGCCAT
CTATATGGTCGGGCC
ATATCATGTCGGGTT
ATATTGAGTCGGGTT
ATATAGATTCGGGTT
ATATACATTCGGGTT
Wall time: 1min 40s


In [44]:
tProf = makeProfile(['CCA', 'CCT', 'CTT', 'TTG'])
dna = ['AAGCCAAA',
       'AATCCTGG',
       'GCTACTTG',
       'ATGTTTTG']
for seq in dna:
    print(profileMost(seq, 3, tProf))

CCA
CCT
CTT
TTT


In [17]:
with open("./Data/DosR.txt") as inFile:
    data = inFile.readlines()

dna = []
for string in data:
    dna.append(string.strip())

# randMotifSrch = dict()
# for k in range(8, 13):
#     tMotif = randMotifDrive(dna, k, len(dna), 50)
#     tScore = score(tMotif)
#     randMotifSrch[k] = (makeCon(makeProfile(tMotif)), tScore)
# print(randMotifSrch)

tMotif = randMotifDrive(dna, 20, len(dna), 75)
tProf = makeProfile(tMotif)
con1 = makeCon(tProf)
print(f"RandomMotifSearch consesus seq: {con1} Score: {score(tMotif)}\n")

tMotif = gibbsDrive(dna, 20, len(dna), 200)
tProf = makeProfile(tMotif)
con2 = makeCon(tProf)
print(f"Gibbs consesus seq: {con2} Score: {score(tMotif)}\n")

RandomMotifSearch consesus seq: GGGGACTTACGACCCTAGCC Score: 62

Gibbs consesus seq: GGGGACCTACGTCCCTAGCC Score: 55

